In [173]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
#%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

In [174]:
'''
# Define path to files
# For Linux
BASE_PATH = "../../../Documents/finance_files/"
DATA_PATH = BASE_PATH
STOCKS_PATH = BASE_PATH +"stocks/"

# Start date defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2017-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2021
E_MONTH = 8
E_DAY = 19
E_DATE_STR = "2021-08-19"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)
'''

'\n# Define path to files\n# For Linux\nBASE_PATH = "../../../Documents/finance_files/"\nDATA_PATH = BASE_PATH\nSTOCKS_PATH = BASE_PATH +"stocks/"\n\n# Start date defaults\nS_YEAR = 2017\nS_MONTH = 1\nS_DAY = 3\nS_DATE_STR = "2017-01-03"\nS_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)\n\n# End date defaults\nE_YEAR = 2021\nE_MONTH = 8\nE_DAY = 19\nE_DATE_STR = "2021-08-19"\nE_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)\n'

In [175]:
#def datestring_to_dates(target_edate):
#    return 

In [176]:
stocks_not_downloaded = []
missing_stocks = []

In [177]:
def get_tickers(stocks_path):
    try:
        df = pd.read_csv(stocks_path + 'stocks.csv')
        print(stocks_path)
    except FileNotFoundError:
        pass
        print("Stocks File Doesn't Exist",stocks_path)
    else:
        return df
#tickers = get_tickers()['ticker'].tolist()
#tickers.sort()
#len(tickers)
#print(tickers)

In [178]:
def delete_unnamed_cols(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

In [179]:
def save_dataframe_to_csv(df, folder, ticker):
    df = delete_unnamed_cols(df)
    df.to_csv(folder + ticker + '.csv')

In [180]:
# Function that gets a dataframe by providing a ticker and starting date
def create_csv_from_yahoo(ticker, syear, smonth, sday, etarget,data_path):
    # Defines the time periods to use
    start = dt.datetime(syear, smonth, sday)
    end = etarget# dt.datetime(eyear, emonth, eday)
    
    try:
        print("Get Data for : ", ticker)
        # Reads data into a dataframe
        #df = web.DataReader(ticker, 'yahoo', start, end)['Adj Close']
        df = yf.download(ticker, start, end)['Adj Close']
        df = df.reset_index()
        df['Date'] = pd.to_datetime(df['Date'])
    
        # Wait 10 seconds
        time.sleep(5)
    
        # Save data to a CSV file linux
        save_dataframe_to_csv(df, data_path, ticker)
        return df
    except Exception as ex:
        stocks_not_downloaded.append(ticker)
        print("Couldn't Get Data for :", ticker)

In [181]:
def get_df_from_csv(folder, ticker):
    try:
        df = pd.read_csv(folder + ticker + '.csv',parse_dates=True)
    except FileNotFoundError:
        pass
        print("File Doesn't Exist")
    else:
        df = delete_unnamed_cols(df)
        return df

In [182]:
def update_stock_data(ticker,data_path,target_edate):
    orig_df = get_df_from_csv(data_path, ticker)
    if orig_df is not None:
        print('there is data',orig_df.shape[0])
        #orig_df = orig_df.reset_index().set_index('Date', drop=False)
        # Drop the unnamed column
        orig_df = delete_unnamed_cols(orig_df)
        orig_df['Date'] = pd.to_datetime(orig_df['Date'])
        # Drop the daily_return column
        if 'daily_return' in orig_df.columns:
            orig_df = orig_df.drop('daily_return',axis = 1, inplace = True)
        #print('old:')
        #display(orig_df.tail(5))
        #check if it needs more data
        edate = pd.to_datetime(orig_df.iloc[-1]['Date'])
        #print(edate)
        if(edate < target_edate):            
            #print(edate,'  ',target_edate)
            updated_df = yf.download(ticker, edate, dt.date.today())['Adj Close']
            if(updated_df is not None):
                updated_df = updated_df.reset_index()
                updated_df['Date'] = pd.to_datetime(updated_df['Date'])
                updated_df = delete_unnamed_cols(updated_df)
                #print('new:',updated_df.shape[0])
                #display(updated_df.head(5))
                #display(updated_df.tail(5))
                orig_df = pd.concat([orig_df[['Date','Adj Close']], updated_df[['Date','Adj Close']]])
                #display(orig_df)
        save_dataframe_to_csv(orig_df, data_path, ticker)
        return orig_df
    else:
        print('there is no data')
        new_df = create_csv_from_yahoo(ticker, 2017, 1, 3, target_edate,data_path)


In [183]:
def update_stocks(stocks_folder,data_path,target_edate):
    tickers = get_tickers(stocks_folder)['ticker'].tolist()

    for x in tickers:
        try:
            print("Working on :", x)
            new_df = update_stock_data(x,data_path,target_edate)
        except Exception as ex:
            print(ex)

In [186]:
#print(get_tickers())
#update_stocks(DATA_PATH)
newdf = update_stock_data('INTC',"../../../Documents/finance_files/",dt.datetime(2020,12,1))

there is data 1914
old:


,Date,Adj Close
1909,2024-08-05,19.983234
1910,2024-08-06,19.705000
1911,2024-08-07,18.990000
1912,2024-08-08,20.490000
1913,2024-08-09,19.709999


2024-08-09 00:00:00
